In [1]:
import argparse
import os
import torch
import torch.nn as nn
import numpy as np
import datetime
from utils.dataset import get_data_loader_folder
from utils.utils import prepare_sub_folder, write_2images, write_html, print_params, adjust_learning_rate
from utils.MattingLaplacian import laplacian_loss_grad

In [2]:
torch.backends.cudnn.benchmark = True
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--base_name', default=None, help='Directory name to save')
parser.add_argument('--mode', type=str, default='photorealistic')
parser.add_argument('--vgg_ckpoint', type=str, default='checkpoints/vgg_normalised.pth')

# Dataset
parser.add_argument('--train_content', default='/home/lexx/ComputerVision/project-4/content/', help='Directory to dataset A')
parser.add_argument('--train_style', default='/home/lexx/ComputerVision/project-4/style/', help='Directory to dataset B')
parser.add_argument('--batch_size', type=int, default=2)
parser.add_argument('--new_size', type=int, default=512)
parser.add_argument('--crop_size', type=int, default=256)

parser.add_argument('--use_lap', type=bool, default=True)
parser.add_argument('--win_rad', type=int, default=1, help='The larger the value, the more detail in the generated image and the higher the CPU and memory requirements (proportional to the win_rad**2)')

# Training options
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--lr_decay', type=float, default=5e-5)

parser.add_argument('--style_weight', type=float, default=1)
parser.add_argument('--content_weight', type=float, default=0)
parser.add_argument('--lap_weight', type=float, default=1500)
parser.add_argument('--rec_weight', type=float, default=10)
parser.add_argument('--temporal_weight', type=float, default=60)

parser.add_argument('--training_iterations', type=int, default=160000)
parser.add_argument('--fine_tuning_iterations', type=int, default=10000)
parser.add_argument("--resume", action="store_true", default=False)
parser.add_argument('--resume_iter', type=int, default=-1)

# Log
parser.add_argument('--logs_directory', default='logs', help='Directory to log')
parser.add_argument('--display_size', type=int, default=16)
parser.add_argument('--image_display_iter', type=int, default=1000)
parser.add_argument('--image_save_iter', type=int, default=10000)
parser.add_argument('--model_save_interval', type=int, default=10000)

_StoreAction(option_strings=['--model_save_interval'], dest='model_save_interval', nargs=None, const=None, default=10000, type=<class 'int'>, choices=None, help=None, metavar=None)

In [4]:
args = parser.parse_args(args=[])

In [5]:
if args.base_name is None:
    args.base_name = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
total_iterations = args.training_iterations + args.fine_tuning_iterations
current_iter = -1


# Logs directory
logs_directory = os.path.join(args.logs_directory, args.base_name)
print("Logs directory:", logs_directory)
checkpoint_directory, image_directory = prepare_sub_folder(logs_directory)

Logs directory: logs/2025-06-15T17-03-47
Creating directory: logs/2025-06-15T17-03-47/images
Creating directory: logs/2025-06-15T17-03-47/checkpoints


In [6]:
# Dataset
batch_size = args.batch_size
num_workers = args.batch_size
new_size = args.new_size
crop_size = args.crop_size
win_rad = args.win_rad
train_loader_a = get_data_loader_folder(args.train_content, batch_size, new_size, crop_size, crop_size, use_lap=True, win_rad=win_rad)
train_loader_b = get_data_loader_folder(args.train_style, batch_size, new_size, crop_size, crop_size, use_lap=False)

In [7]:
# Reversible Network
from models.RevResNet import RevResNet
if args.mode.lower() == "photorealistic":
    RevNetwork = RevResNet(nBlocks=[10, 10, 10], nStrides=[1, 2, 2], nChannels=[16, 64, 256], in_channel=3, mult=4, hidden_dim=16, sp_steps=2)
elif args.mode.lower() == "artistic":
    RevNetwork = RevResNet(nBlocks=[10, 10, 10], nStrides=[1, 2, 2], nChannels=[16, 64, 256], in_channel=3, mult=4, hidden_dim=64, sp_steps=1)
else:
    raise NotImplementedError()

RevNetwork = RevNetwork.to(device)
RevNetwork.train()
print_params(RevNetwork)


 == Building Reversible Residual Network 91 Blocks== 
Total params: 4089936
Trainable params: 4089936
Non-trainable params: 0


In [8]:
# Optimizer
optimizer = torch.optim.Adam(RevNetwork.parameters(), lr=args.lr)


# Transfer module
from models.cWCT import cWCT
cwct = cWCT(train_mode=True)


# VGG for style loss
from models.VGG import VGG19
vgg_enc = VGG19(args.vgg_ckpoint)
# encoder = nn.DataParallel(encoder)
vgg_enc.to(device)

VGG19(
  (enc_1): Sequential(
    (0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (1): ReflectionPad2d((1, 1, 1, 1))
    (2): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (enc_2): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=True)
    (4): ReflectionPad2d((1, 1, 1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
  )
  (enc_3): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=True)
    (4): ReflectionPad2d((1, 1, 1, 1))
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
  )
  (enc_4): Sequential(
    (0): ReflectionPad2d((1, 1, 1, 1))
    (1): Conv2d(256, 25

In [9]:
# Resume
if args.resume:
    state_dict = torch.load(os.path.join(checkpoint_directory, "last.pt"))
    RevNetwork.load_state_dict(state_dict['state_dict'])
    optimizer.load_state_dict(state_dict['optimizer'])
    current_iter = args.resume_iter
    print('Resume from %s. Resume iter is %d' % (os.path.join(checkpoint_directory, "last.pt"), args.resume_iter))


# Loss
l1_loss = torch.nn.L1Loss()
if args.temporal_weight > 0:
    from utils.TemporalLoss import TemporalLoss
    Temporal_loss = TemporalLoss()

In [10]:
# Training
iter_loader_a, iter_loader_b = iter(train_loader_a), iter(train_loader_b)
while current_iter < total_iterations:
    images_a, images_b = next(iter_loader_a), next(iter_loader_b)

    lap_list = []
    if args.lap_weight > 0:
        for M in images_a['laplacian_m']:
            indices = torch.from_numpy(np.vstack((M.row, M.col))).long().to(device)
            values = torch.from_numpy(M.data).to(device)
            shape = torch.Size(M.shape)
            laplacian_m = torch.sparse_coo_tensor(indices, values, shape, device=device)
            lap_list.append(laplacian_m)

    images_a, images_b = images_a['img'].to(device), images_b['img'].to(device)

    # Optimizer
    adjust_learning_rate(optimizer, args.lr, args.lr_decay, current_iter)
    optimizer.zero_grad()

    # Forward inference
    z_c = RevNetwork(images_a, forward=True)
    z_s = RevNetwork(images_b, forward=True)

    # Transfer
    try:
        z_cs = cwct.transfer(z_c, z_s)
    except:
        print('Cholesky Decomposition fails. Gradient infinity. Skip current batch.')
        with open(logs_directory + "/loss.log", "a") as log_file:
            log_file.write('Cholesky Decomposition fails. Gradient infinity. Skip current batch. \n')
        continue

    # Backward inference
    stylized = RevNetwork(z_cs, forward=False)


    # Style loss
    loss_c, loss_s = vgg_enc(images_a, images_b, stylized, n_layer=4, content_weight=args.content_weight)

    # Cycle reconstruction
    if args.rec_weight > 0:
        z_cs = RevNetwork(stylized, forward=True)

        try:
            z_csc = cwct.transfer(z_cs, z_c)
        except:
            print('Cholesky Decomposition fails. Gradient infinity. Skip current batch.')
            with open(logs_directory + "/loss.log", "a") as log_file:
                log_file.write('Cholesky Decomposition fails. Gradient infinity. Skip current batch. \n')
            continue

        rec = RevNetwork(z_csc, forward=False)
        loss_rec = l1_loss(rec, images_a)
    else:
        loss_rec = 0

    # Matting Laplacian loss
    if args.lap_weight > 0:
        bn = stylized.size(0)
        lap_loss = list()
        grad = list()
        for i in range(bn):
            l, g = laplacian_loss_grad(stylized[i], lap_list[i])
            lap_loss.append(l)
            grad.append(g)
        grad = torch.stack(grad, dim=0)
        grad = grad * args.lap_weight
        grad = grad.clamp(-0.05, 0.05)
        stylized.backward(grad, retain_graph=True)  # We can directly backward gradient

        loss_lap = torch.mean(torch.stack(lap_loss, dim=0))
    else:
        loss_lap = 0

    # Temporal loss
    if args.temporal_weight > 0 and current_iter > args.training_iterations:
        SecondFrame, ForwardFlow = Temporal_loss.GenerateFakeData(images_a)
        z_c2 = RevNetwork(SecondFrame, forward=True)

        try:
            z_cs2 = cwct.transfer(z_c2, z_s)
        except:
            print('Cholesky Decomposition fails. Gradient infinity. Skip current batch.')
            with open(logs_directory + "/loss.log", "a") as log_file:
                log_file.write('Cholesky Decomposition fails. Gradient infinity. Skip current batch. \n')
            continue

        stylizedSecondFrame = RevNetwork(z_cs2, forward=False)

        loss_tmp, FakeStyledSecondFrame_1 = Temporal_loss(stylized, stylizedSecondFrame, ForwardFlow)

        loss_tmp_GT, _ = Temporal_loss(images_a, SecondFrame, ForwardFlow)
    else:
        loss_tmp = 0
        loss_tmp_GT = 0.


    # Total loss
    loss = args.content_weight * loss_c + args.style_weight * loss_s + args.rec_weight * loss_rec + args.temporal_weight * loss_tmp

    loss.backward()
    nn.utils.clip_grad_norm_(RevNetwork.parameters(), 5)
    optimizer.step()


    # Dump training stats in log file
    if (current_iter + 1) % 10 == 0:
        message = "Iteration: %08d/%08d  content_loss:%.4f  lap_loss:%.4f  rec_loss:%.4f  style_loss:%.4f  loss_tmp:%.4f  loss_tmp_GT:%.4f" % (
            current_iter + 1, total_iterations,
            args.content_weight * loss_c,
            args.lap_weight * loss_lap,
            args.rec_weight * loss_rec,
            args.style_weight * loss_s,
            args.temporal_weight * loss_tmp,
            args.temporal_weight * loss_tmp_GT,
        )
        print(message)
        with open(logs_directory + "/loss.log", "a") as log_file:
            log_file.write('%s\n' % message)

        # Log sample
        if (current_iter + 1) % args.image_save_iter == 0:
            cwct.train_mode = False
            with torch.no_grad():
                index = torch.randint(low=0, high=len(train_loader_a.dataset), size=[args.display_size])
                train_display_images_a = torch.stack([train_loader_a.dataset[i]['img'] for i in index])
                index = torch.randint(low=0, high=len(train_loader_b.dataset), size=[args.display_size])
                train_display_images_b = torch.stack([train_loader_b.dataset[i]['img'] for i in index])
                train_image_outputs = RevNetwork.sample(cwct, train_display_images_a, train_display_images_b, device)
            cwct.train_mode = True
            write_2images(train_image_outputs, args.display_size, image_directory, 'train_%08d' % (current_iter + 1))
            # HTML
            write_html(logs_directory + "/index.html", current_iter + 1, args.image_save_iter, 'images')

        if (current_iter + 1) % args.image_display_iter == 0:
            cwct.train_mode = False
            with torch.no_grad():
                index = torch.randint(low=0, high=len(train_loader_a.dataset), size=[args.display_size])
                train_display_images_a = torch.stack([train_loader_a.dataset[i]['img'] for i in index])
                index = torch.randint(low=0, high=len(train_loader_b.dataset), size=[args.display_size])
                train_display_images_b = torch.stack([train_loader_b.dataset[i]['img'] for i in index])
                image_outputs = RevNetwork.sample(cwct, train_display_images_a, train_display_images_b, device)
            cwct.train_mode = True
            write_2images(image_outputs, args.display_size, image_directory, 'train_current')

        # Save network weights
        if (current_iter + 1) % args.model_save_interval == 0:
            ckpoint_file = os.path.join(checkpoint_directory, 'last.pt')
            torch.save({'state_dict': RevNetwork.state_dict(), 'optimizer': optimizer.state_dict()}, ckpoint_file)

        if (current_iter + 1) == args.training_iterations:
            ckpoint_file = os.path.join(checkpoint_directory, 'model_image.pt')
            torch.save({'state_dict': RevNetwork.state_dict()}, ckpoint_file)
        elif (current_iter + 1) == total_iterations:
            ckpoint_file = os.path.join(checkpoint_directory, 'model_video.pt')
            torch.save({'state_dict': RevNetwork.state_dict()}, ckpoint_file)

    current_iter += 1

print("Finishing training. Model save at %s" % checkpoint_directory)

Iteration: 00000000/00170000  content_loss:0.0000  lap_loss:13.9266  rec_loss:0.2416  style_loss:3.7925  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000010/00170000  content_loss:0.0000  lap_loss:16.1721  rec_loss:0.1536  style_loss:6.4365  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000020/00170000  content_loss:0.0000  lap_loss:29.4093  rec_loss:0.0857  style_loss:3.7552  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000030/00170000  content_loss:0.0000  lap_loss:4.3187  rec_loss:0.1382  style_loss:2.0149  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000040/00170000  content_loss:0.0000  lap_loss:55.3806  rec_loss:0.1088  style_loss:8.5740  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000050/00170000  content_loss:0.0000  lap_loss:4.1838  rec_loss:0.2041  style_loss:8.3018  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000060/00170000  content_loss:0.0000  lap_loss:4.8747  rec_loss:0.1206  style_loss:3.1470  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000

Iteration: 00000590/00170000  content_loss:0.0000  lap_loss:0.2476  rec_loss:0.1277  style_loss:3.0464  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000600/00170000  content_loss:0.0000  lap_loss:0.1745  rec_loss:0.1489  style_loss:2.9116  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000610/00170000  content_loss:0.0000  lap_loss:0.5904  rec_loss:0.2295  style_loss:5.9370  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000620/00170000  content_loss:0.0000  lap_loss:0.2916  rec_loss:0.0712  style_loss:2.2644  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000630/00170000  content_loss:0.0000  lap_loss:0.3350  rec_loss:0.1254  style_loss:2.5247  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000640/00170000  content_loss:0.0000  lap_loss:0.2308  rec_loss:0.0725  style_loss:2.2064  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000650/00170000  content_loss:0.0000  lap_loss:1.3357  rec_loss:0.2167  style_loss:3.7007  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00000660/

Iteration: 00001180/00170000  content_loss:0.0000  lap_loss:0.4164  rec_loss:0.1111  style_loss:3.5782  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001190/00170000  content_loss:0.0000  lap_loss:0.1362  rec_loss:0.0994  style_loss:2.5187  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001200/00170000  content_loss:0.0000  lap_loss:0.8653  rec_loss:0.0949  style_loss:6.2564  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001210/00170000  content_loss:0.0000  lap_loss:0.1399  rec_loss:0.0916  style_loss:2.8035  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001220/00170000  content_loss:0.0000  lap_loss:0.2975  rec_loss:0.2637  style_loss:4.5948  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001230/00170000  content_loss:0.0000  lap_loss:0.0939  rec_loss:0.1184  style_loss:1.1722  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001240/00170000  content_loss:0.0000  lap_loss:0.0650  rec_loss:0.0497  style_loss:1.8320  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001250/

Iteration: 00001770/00170000  content_loss:0.0000  lap_loss:0.1451  rec_loss:0.3467  style_loss:2.7227  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001780/00170000  content_loss:0.0000  lap_loss:0.4540  rec_loss:0.0888  style_loss:3.1214  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001790/00170000  content_loss:0.0000  lap_loss:0.5650  rec_loss:0.1079  style_loss:10.2148  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001800/00170000  content_loss:0.0000  lap_loss:0.0832  rec_loss:0.1264  style_loss:1.5173  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001810/00170000  content_loss:0.0000  lap_loss:0.1648  rec_loss:0.1456  style_loss:1.6224  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001820/00170000  content_loss:0.0000  lap_loss:0.2729  rec_loss:0.1222  style_loss:2.9798  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001830/00170000  content_loss:0.0000  lap_loss:0.4122  rec_loss:0.1336  style_loss:7.5546  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00001840

Iteration: 00002360/00170000  content_loss:0.0000  lap_loss:0.2730  rec_loss:0.1496  style_loss:3.2836  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002370/00170000  content_loss:0.0000  lap_loss:0.1975  rec_loss:0.1697  style_loss:3.1326  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002380/00170000  content_loss:0.0000  lap_loss:0.0936  rec_loss:0.0868  style_loss:1.2850  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002390/00170000  content_loss:0.0000  lap_loss:0.1650  rec_loss:0.1092  style_loss:3.7058  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002400/00170000  content_loss:0.0000  lap_loss:0.0548  rec_loss:0.1549  style_loss:1.3750  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002410/00170000  content_loss:0.0000  lap_loss:0.0925  rec_loss:0.2374  style_loss:1.6297  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002420/00170000  content_loss:0.0000  lap_loss:0.4042  rec_loss:0.1184  style_loss:5.5693  loss_tmp:0.0000  loss_tmp_GT:0.0000
Iteration: 00002430/

...

Iteration: 00167090/00170000  content_loss:0.0000  lap_loss:0.2521  rec_loss:0.0981  style_loss:1.4267  loss_tmp:0.2797  loss_tmp_GT:0.0896
Iteration: 00167100/00170000  content_loss:0.0000  lap_loss:0.0552  rec_loss:0.2391  style_loss:0.8731  loss_tmp:0.1502  loss_tmp_GT:0.0695
Iteration: 00167110/00170000  content_loss:0.0000  lap_loss:0.0455  rec_loss:0.0489  style_loss:0.7137  loss_tmp:0.3197  loss_tmp_GT:0.0496
Iteration: 00167120/00170000  content_loss:0.0000  lap_loss:0.0435  rec_loss:0.0738  style_loss:0.7045  loss_tmp:0.3045  loss_tmp_GT:0.0807
Iteration: 00167130/00170000  content_loss:0.0000  lap_loss:0.0894  rec_loss:0.1008  style_loss:1.5248  loss_tmp:0.2890  loss_tmp_GT:0.0788
Iteration: 00167140/00170000  content_loss:0.0000  lap_loss:0.2827  rec_loss:0.0908  style_loss:1.4893  loss_tmp:0.3255  loss_tmp_GT:0.0590
Iteration: 00167150/00170000  content_loss:0.0000  lap_loss:0.0493  rec_loss:0.0715  style_loss:0.8184  loss_tmp:0.2727  loss_tmp_GT:0.0483
Iteration: 00167160/

Iteration: 00167680/00170000  content_loss:0.0000  lap_loss:0.0803  rec_loss:0.0769  style_loss:1.5159  loss_tmp:0.3990  loss_tmp_GT:0.0550
Iteration: 00167690/00170000  content_loss:0.0000  lap_loss:0.0931  rec_loss:0.3129  style_loss:1.3938  loss_tmp:0.2283  loss_tmp_GT:0.0546
Iteration: 00167700/00170000  content_loss:0.0000  lap_loss:0.1635  rec_loss:0.0640  style_loss:4.0154  loss_tmp:0.4167  loss_tmp_GT:0.0891
Iteration: 00167710/00170000  content_loss:0.0000  lap_loss:0.1215  rec_loss:0.1011  style_loss:1.8178  loss_tmp:0.4445  loss_tmp_GT:0.0666
Iteration: 00167720/00170000  content_loss:0.0000  lap_loss:0.1189  rec_loss:0.0634  style_loss:2.1671  loss_tmp:0.3594  loss_tmp_GT:0.0874
Iteration: 00167730/00170000  content_loss:0.0000  lap_loss:0.0961  rec_loss:0.0940  style_loss:2.4687  loss_tmp:0.6025  loss_tmp_GT:0.0757
Iteration: 00167740/00170000  content_loss:0.0000  lap_loss:0.0490  rec_loss:0.0514  style_loss:1.2330  loss_tmp:0.2116  loss_tmp_GT:0.0752
Iteration: 00167750/

Iteration: 00168270/00170000  content_loss:0.0000  lap_loss:0.0543  rec_loss:0.0825  style_loss:0.8794  loss_tmp:0.3807  loss_tmp_GT:0.0484
Iteration: 00168280/00170000  content_loss:0.0000  lap_loss:0.0518  rec_loss:0.0613  style_loss:1.5819  loss_tmp:0.6007  loss_tmp_GT:0.0890
Iteration: 00168290/00170000  content_loss:0.0000  lap_loss:0.1031  rec_loss:0.0742  style_loss:1.2253  loss_tmp:0.3610  loss_tmp_GT:0.0894
Iteration: 00168300/00170000  content_loss:0.0000  lap_loss:0.0525  rec_loss:0.0592  style_loss:1.2430  loss_tmp:0.2968  loss_tmp_GT:0.0568
Iteration: 00168310/00170000  content_loss:0.0000  lap_loss:0.1369  rec_loss:0.1147  style_loss:2.2057  loss_tmp:0.6139  loss_tmp_GT:0.0628
Iteration: 00168320/00170000  content_loss:0.0000  lap_loss:0.2939  rec_loss:0.0971  style_loss:2.3652  loss_tmp:0.5167  loss_tmp_GT:0.0570
Iteration: 00168330/00170000  content_loss:0.0000  lap_loss:0.0382  rec_loss:0.1842  style_loss:0.5482  loss_tmp:0.1480  loss_tmp_GT:0.0484
Iteration: 00168340/

Iteration: 00168860/00170000  content_loss:0.0000  lap_loss:0.0460  rec_loss:0.0819  style_loss:0.9169  loss_tmp:0.2717  loss_tmp_GT:0.0809
Iteration: 00168870/00170000  content_loss:0.0000  lap_loss:0.0521  rec_loss:0.0988  style_loss:1.2078  loss_tmp:0.6433  loss_tmp_GT:0.0818
Iteration: 00168880/00170000  content_loss:0.0000  lap_loss:0.2542  rec_loss:0.0959  style_loss:2.6921  loss_tmp:0.4155  loss_tmp_GT:0.0791
Iteration: 00168890/00170000  content_loss:0.0000  lap_loss:0.0578  rec_loss:0.1428  style_loss:1.5062  loss_tmp:0.3805  loss_tmp_GT:0.0576
Iteration: 00168900/00170000  content_loss:0.0000  lap_loss:0.0747  rec_loss:0.0697  style_loss:2.2989  loss_tmp:0.6149  loss_tmp_GT:0.0709
Iteration: 00168910/00170000  content_loss:0.0000  lap_loss:0.0483  rec_loss:0.1307  style_loss:0.8444  loss_tmp:0.2163  loss_tmp_GT:0.0761
Iteration: 00168920/00170000  content_loss:0.0000  lap_loss:0.0337  rec_loss:0.1747  style_loss:0.5581  loss_tmp:0.1104  loss_tmp_GT:0.0718
Iteration: 00168930/

Iteration: 00169450/00170000  content_loss:0.0000  lap_loss:0.0484  rec_loss:0.0986  style_loss:0.5737  loss_tmp:0.1938  loss_tmp_GT:0.0611
Iteration: 00169460/00170000  content_loss:0.0000  lap_loss:0.1986  rec_loss:0.2608  style_loss:2.6615  loss_tmp:0.2664  loss_tmp_GT:0.0807
Iteration: 00169470/00170000  content_loss:0.0000  lap_loss:0.0810  rec_loss:0.0920  style_loss:1.8632  loss_tmp:0.2022  loss_tmp_GT:0.0494
Iteration: 00169480/00170000  content_loss:0.0000  lap_loss:0.2382  rec_loss:0.0497  style_loss:3.6005  loss_tmp:0.8498  loss_tmp_GT:0.0645
Iteration: 00169490/00170000  content_loss:0.0000  lap_loss:0.0428  rec_loss:0.1062  style_loss:0.9919  loss_tmp:0.2399  loss_tmp_GT:0.0763
Iteration: 00169500/00170000  content_loss:0.0000  lap_loss:0.0389  rec_loss:0.0746  style_loss:0.7476  loss_tmp:0.2864  loss_tmp_GT:0.0842
Iteration: 00169510/00170000  content_loss:0.0000  lap_loss:0.0594  rec_loss:0.0600  style_loss:1.0033  loss_tmp:0.3809  loss_tmp_GT:0.0886
Iteration: 00169520/

In [52]:
opt.cuda

True